In [1]:
%cd ..

/workspaces/RAG-Flash-card-app


/home/vscode/.local/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
from src.ingest import *
from src.generator import generate_flashcard, generate_question_answering
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
raw_texts = load_documents('./data/python_tut.pdf')
raw_texts

[Document(metadata={'producer': 'xdvipdfmx (0.7.9)', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-09-02T00:46:10+00:00', 'author': 'Guido van Rossum, and the Python development team', 'moddate': '2018-09-02T11:19:20-07:00', 'title': 'Python Tutorial', 'source': './data/python_tut.pdf', 'total_pages': 155, 'page': 0, 'page_label': '1'}, page_content='Python Tutorial\nRelease 3.7.0\nGuido van Rossum\nand the Python development team\nSeptember 02, 2018\nPython Software Foundation\nEmail: docs@python.org'),
 Document(metadata={'producer': 'xdvipdfmx (0.7.9)', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-09-02T00:46:10+00:00', 'author': 'Guido van Rossum, and the Python development team', 'moddate': '2018-09-02T11:19:20-07:00', 'title': 'Python Tutorial', 'source': './data/python_tut.pdf', 'total_pages': 155, 'page': 1, 'page_label': '2'}, page_content=''),
 Document(metadata={'producer': 'xdvipdfmx (0.7.9)', 'creator': 'LaTeX with hyperref package', 'c

In [3]:
texts = chunk_text(raw_texts)
len(texts)

304

In [ ]:
embedding_function = get_embedder(api_key, "text-embedding-3-small")
print(embedding_function)
collection = create_vectorstore(
    file_name="python_tut.pdf",
    vectorstore_path='./models/embeddings/chroma_db',
    embedding_function=embedding_function, 
    texts=texts) 

client=<openai.resources.embeddings.Embeddings object at 0xffff61461520> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0xffff6133b3b0> model='text-embedding-3-small' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True


In [ ]:
embedding_function = get_embedder(api_key)
vectorstore = load_vectorstore(
    file_name="python_tut.pdf",
    vectorstore_path='./models/embeddings/chroma_db',
    embedding_function=embedding_function) 


retriever = vectorstore.as_retriever(search_type='similarity')
context = retriever.invoke("What is Intermezzo: Coding Style")
context

In [11]:
res = generate_flashcard(context, 3, api_key)

/home/vscode/.local/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1360: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


In [23]:
res.values[0]

array([list([{'question': 'What is the recommended indentation style in Python?', 'keypoint': 'Use 4-space indentation, and no tabs.', 'sources': 'For Python, PEP 8 has emerged as the style guide that most projects adhere to; it promotes a very readable and eye-pleasing coding style.', 'reasoning': 'This statement is a key recommendation from PEP 8, which is the style guide for Python that encourages readability in coding.'}, {'question': 'What is the maximum line length recommended for Python code?', 'keypoint': 'Wrap lines so that they don’t exceed 79 characters.', 'sources': 'Wrap lines so that they don’t exceed 79 characters. This helps users with small displays and makes it possible to have several code ﬁles side-by-side on larger displays.', 'reasoning': 'This guideline aims to enhance the readability of code and is specifically mentioned in the context of accommodating users with different display sizes.'}, {'question': 'What style does the LBYL coding style represent?', 'keypoi

In [ ]:
res.values[0][0][0]['question'], res.values[0][0][0]['keypoint']

('What is the recommended indentation style in Python?',
 'Use 4-space indentation, and no tabs.')

In [25]:
res2 = generate_question_answering(context, 'What is Intermezzo: Coding Style', api_key)

/home/vscode/.local/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1360: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


In [26]:
res2

AnswerWithSources(answer='Intermezzo: Coding Style is a section in the Python tutorial that discusses the importance of adopting a clear and readable coding style when writing Python code. It emphasizes that making code easy to read is beneficial for collaboration and maintenance, and it primarily refers to PEP 8, which is the style guide that suggests best practices for writing Python code. Key recommendations from this section include using 4-space indentation, avoiding tabs, wrapping lines to not exceed 79 characters, separating functions and classes with blank lines, placing comments on separate lines when possible, and using docstrings for documentation.', sources='For Python, PEP 8 has emerged as the style guide that most projects adhere to; it promotes a very readable and eye-pleasing coding style. ... Here are the most important points extracted for you: • Use 4-space indentation, and no tabs. ... • Use blank lines to separate functions and classes, and larger blocks of code in

In [27]:
res2.answer, res2.sources

('Intermezzo: Coding Style is a section in the Python tutorial that discusses the importance of adopting a clear and readable coding style when writing Python code. It emphasizes that making code easy to read is beneficial for collaboration and maintenance, and it primarily refers to PEP 8, which is the style guide that suggests best practices for writing Python code. Key recommendations from this section include using 4-space indentation, avoiding tabs, wrapping lines to not exceed 79 characters, separating functions and classes with blank lines, placing comments on separate lines when possible, and using docstrings for documentation.',
 'For Python, PEP 8 has emerged as the style guide that most projects adhere to; it promotes a very readable and eye-pleasing coding style. ... Here are the most important points extracted for you: • Use 4-space indentation, and no tabs. ... • Use blank lines to separate functions and classes, and larger blocks of code inside functions. • When possible